<a href="https://colab.research.google.com/github/marcelpinheiro/malum/blob/master/avro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://malum.com.br/wp/2019/07/09/avro/

#1
!pip install avro-python3
!pip install requests
!pip install pandas

  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=603850d4e436f63389c31bd9ce283c17231cafcbbb7bedb87c3b7a67ae0fb77a
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
Successfully built avro-python3


In [2]:
#2
import requests
import pandas as pd


dataRequest = requests.get(url='http://api.tvmaze.com/singlesearch/shows?q=game-of-thrones&embed=episodes')
data_json = dataRequest.json()

from pandas.io.json import json_normalize

data_normalize = json_normalize(
                                    #Dados deserializados
                                    #Unserialized JSON objects
                                    data=data_json['_embedded']
    
                                    #Caminho do objeto aonde estão os dados
                                    #path of the data that you need
                                    ,record_path='episodes'
                                    #Prefixo das colunas. Recomendado usar para evitar ambiguidade de colunas
                                    # Columns prefix. Recommended to use to avoid ambiguous name columns                                    
                                    ,record_prefix='got_'
                                    )
#Listaremos apenas as colunas que precisamos
#Listing only the columns that we will need
data = data_normalize[['got_season','got_number','got_name']]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [3]:
#3
#Usamos 3 aspas duplas para tudo virar string / We use triple doouble quotes to everything inside be string
schema = """{
  "type": "record",
  "name": "GOTLIST",
  "fields" : [
    {"name": "got_season", "type": "int", "doc": "Número da temporada / Season number"},             
    {"name": "got_number", "type": "int", "doc": "Número do episódio / Episode number"},             
    {"name": "got_name", "type": "string", "doc": "Título do episódio / Episode title"}             

  ]
}"""

In [4]:
#4
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import pandas as pd

schemaParse = avro.schema.Parse(schema)

#Aqui nós criaremos um arquivo com a lista de episódios com o schema para validação
#Here we create a file of GOT episodes list with the schema for validation
writer = DataFileWriter(open("got_malum.avro", "wb"), DatumWriter(), schemaParse)

# writer.append({"got_season": 1, "got_number": 1, "got_name": "Test"})
#Agora iremos inserir os dados da variavél data no arquivo got_malum.avro
#Now we will insert the data from the data variable in the got_malum.avro file

for index, rows in data.iterrows():

    writer.append({"got_season": rows['got_season'], "got_number": rows['got_number'], "got_name": rows['got_name']})


writer.close()


In [5]:
#Lendo o arquivo (deserializar) / reading the file (Deserialization)
reader = DataFileReader(open("got_malum.avro", "rb"), DatumReader())
for user in reader:
    print(user)
reader.close()

{'got_season': 1, 'got_number': 1, 'got_name': 'Winter is Coming'}
{'got_season': 1, 'got_number': 2, 'got_name': 'The Kingsroad'}
{'got_season': 1, 'got_number': 3, 'got_name': 'Lord Snow'}
{'got_season': 1, 'got_number': 4, 'got_name': 'Cripples, Bastards, and Broken Things'}
{'got_season': 1, 'got_number': 5, 'got_name': 'The Wolf and the Lion'}
{'got_season': 1, 'got_number': 6, 'got_name': 'A Golden Crown'}
{'got_season': 1, 'got_number': 7, 'got_name': 'You Win or You Die'}
{'got_season': 1, 'got_number': 8, 'got_name': 'The Pointy End'}
{'got_season': 1, 'got_number': 9, 'got_name': 'Baelor'}
{'got_season': 1, 'got_number': 10, 'got_name': 'Fire and Blood'}
{'got_season': 2, 'got_number': 1, 'got_name': 'The North Remembers'}
{'got_season': 2, 'got_number': 2, 'got_name': 'The Night Lands'}
{'got_season': 2, 'got_number': 3, 'got_name': 'What is Dead May Never Die'}
{'got_season': 2, 'got_number': 4, 'got_name': 'Garden of Bones'}
{'got_season': 2, 'got_number': 5, 'got_name': '